In [142]:
# This cell imports stuff, and sets up a bot instance etc

import sys
import lzma
from s2clientprotocol.sc2api_pb2 import Response, ResponseObservation
from MapAnalyzer.MapData import MapData
import pickle
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../src")
sys.path.append("../src/ares")
from sc2.position import Point2
from sc2.client import Client
from sc2.game_data import GameData
from sc2.game_info import GameInfo
from sc2.game_state import GameState
from unittest.mock import patch
from ares import AresBot
from ares.dicts.unit_data import UNIT_DATA
from sc2.bot_ai import BotAI

async def build_bot_object_from_pickle_data(raw_game_data, raw_game_info, raw_observation) -> AresBot:
    # Build fresh bot object, and load the pickled data into the bot object
    bot = BotAI()
    game_data = GameData(raw_game_data.data)
    game_info = GameInfo(raw_game_info.game_info)
    game_state = GameState(raw_observation)
    bot._initialize_variables()
    client = Client(True)
    
    bot._prepare_start(client=client, player_id=1, game_info=game_info, game_data=game_data)
    with patch.object(Client, "query_available_abilities_with_tag", return_value={}):
        await bot._prepare_step(state=game_state, proto_game_info=raw_game_info)
        bot._prepare_first_step()
        # await bot.register_managers()
    return bot

BERLINGRAD = "../tests/pickle_data/BerlingradAIE.xz"
with lzma.open(BERLINGRAD, "rb") as f:
    raw_game_data, raw_game_info, raw_observation = pickle.load(f)

# initiate a BotAI and MapAnalyzer instance
bot = await build_bot_object_from_pickle_data(raw_game_data, raw_game_info, raw_observation)
data = MapData(bot)

# common variables
grid = data.get_pyastar_grid()
position = bot.enemy_start_locations[0]
units = bot.all_units

%load_ext line_profiler
%load_ext Cython

2023-06-15 16:49:20.507 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling Berlingrad AIE 
 Version dev Map Compilation Progress : 0.4it [00:00,  1.56it/s]

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# Cythonizing commonly used functions in a python-sc2 bot

Actual speedups many vary as notebook gets rerun

* [Converting is_position_safe](#Python-version-of-is_position_safe) **6.57x speedup**
* [Alternative to python-sc2's units.closest_to](#Alternative-to-python-sc2's-units.closest_to) **6.7x speedup**
* [Speeding up `Units.center`](#Speeding-up-Units.center) **2.01x speedup**
* [Distance to / `unit.distance_to`](#Distance-to-/-unit.distance_to) **2.47x speedup**
* [Convert `already_pending` for units](#Convert-already_pending-for-units) **6.62x speedup**
* [Alternative to `python-sc2`'s `units.in_attack_range`](#Alternative-to-python-sc2's-units.in_attack_range) **4.17x speedup**
* [Calculate if attack is ready](#Calculate-if-attack-is-ready) **3.87x speedup**
* [Pick enemy target](#Pick-enemy-target) **1.63x speedup**

# Converting `is_position_safe` to cython

## Python version of `is_position_safe`

In [143]:
def is_position_safe(
    grid: np.ndarray,
    position: Point2,
    weight_safety_limit: float = 1.0,
) -> bool:
    weight: float = grid[position.rounded]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [144]:
%timeit is_position_safe(grid, position)

3.99 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [145]:
%lprun -f is_position_safe is_position_safe(grid, position)

Timer unit: 1e-07 s

Total time: 4.6e-05 s
File: C:\Users\Tom\AppData\Local\Temp\ipykernel_25596\2777119042.py
Function: is_position_safe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def is_position_safe(
     2                                               grid: np.ndarray,
     3                                               position: Point2,
     4                                               weight_safety_limit: float = 1.0,
     5                                           ) -> bool:
     6         1        127.0    127.0     27.6      weight: float = grid[position.rounded]
     7                                               # np.inf check if drone is pathing near a spore crawler
     8         1        333.0    333.0     72.4      return weight == np.inf or weight <= weight_safety_limit

## Cython version of `is_position_safe`

In [146]:
%%cython
import numpy as np
cimport numpy as cnp
from cython cimport boundscheck, wraparound
@boundscheck(False)
@wraparound(False)
cpdef bint is_position_safe(
    cnp.ndarray[cnp.npy_float32, ndim=2] grid,
    (unsigned int, unsigned int) position,
    double weight_safety_limit = 1.0,
):
    cdef double weight = 0.0
    weight = grid[position[0], position[1]]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [147]:
%timeit is_position_safe(grid, position.rounded)

632 ns ± 4.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Alternative to `python-sc2`'s `units.closest_to`

In [148]:
units = bot.all_units
position = bot.enemy_start_locations[0]
unit = units[0]

In [149]:
# slower using closest_to(Point2)
%timeit units.closest_to(position)

200 µs ± 596 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [150]:
# this is faster since distance between all units is cached
%timeit units.closest_to(unit)

99.3 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [151]:
%%cython
from cython cimport boundscheck, wraparound

cdef double euclidean_distance_squared(
        (float, float) p1,
        (float, float) p2
):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

@boundscheck(False)
@wraparound(False)
cpdef object closest_to((float, float) position, object units):
    cdef:
        object closest = units[0]
        double closest_dist = 999.9
        double dist = 0.0
        unsigned int len_units = len(units)
        (float, float) pos
        
    for i in range(len_units):
        unit = units[i]
        pos = unit.position
        dist = euclidean_distance_squared((pos[0], pos[1]), (position[0], position[1]))
        if dist < closest_dist:
            closest_dist = dist
            closest = unit
            
    return closest


In [152]:
%timeit closest_to(position, units)

15.1 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Speeding up `Units.center`

Similar to `units.center` in `python-sc2`, but tweaked to work in this notebook

In [153]:
from sc2.position import Point2
def center(units) -> Point2:
    """Returns the central position of all units."""
    assert units, f"Units object is empty"
    amount = units.amount
    return Point2(
        (
            sum(unit._proto.pos.x for unit in units) / amount,
            sum(unit._proto.pos.y for unit in units) / amount,
        )
    )

In [154]:
%timeit center(units)

110 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Convert `Units.center` to cython

In [155]:
%%cython

cimport cython
from sc2.position import Point2

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef (double, double) cy_center(object units):
    """Returns the central position of all units."""
    cdef:
        unsigned int i = 0
        unsigned int num_units = len(units)
        double sum_x, sum_y = 0.0
        (double, double) position
        object unit

    for i in range(num_units):
        pos = units[i]._proto.pos
        position = (pos.x, pos.y)
        sum_x += position[0]
        sum_y += position[1]

    return (sum_x / num_units, sum_y / num_units)

In [156]:
%timeit Point2(cy_center(units))

54.4 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Distance to / `unit.distance_to`

Check and profile distances between 2 units / 1 unit and a Point2 / two Point2's using `python-sc2` implementation of `distance_to`. 

Then see if cython does it faster

In [157]:
unit1 = bot.workers[0]
unit2 = bot.workers[4]
position1 = bot.game_info.map_center
position2 = bot.main_base_ramp.top_center

In [158]:
unit1.distance_to(unit2)

5.0

In [159]:
%timeit unit1.distance_to(unit2)

557 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [160]:
unit1.distance_to(position1)

73.81395532011545

In [161]:
%timeit unit1.distance_to(position1)

986 ns ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [162]:
position1.distance_to(position2)

53.766904318548974

In [163]:
%timeit position1.distance_to(position2)

384 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Convert `distance_to` to cython

In [164]:
%%cython

cpdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

In [165]:
cy_distance_to(unit1.position, unit2.position)

5.0

In [166]:
%timeit cy_distance_to(unit1.position, unit2.position)

198 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [167]:
cy_distance_to(unit1.position, position1)

73.81395532011545

In [168]:
%timeit cy_distance_to(unit1.position, position1)

169 ns ± 0.486 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [169]:
cy_distance_to(position1, position2)

53.766907769498424

In [170]:
%timeit cy_distance_to(position1, position2)

154 ns ± 0.387 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


# Convert `already_pending` for units

This seems like a slow function in profiles, make an alternative for units and structures

## `python-sc2`'s `already_pending` function:

In [171]:
from sc2.ids.unit_typeid import UnitTypeId

In [172]:
%timeit bot.already_pending(UnitTypeId.MARINE)

2.55 µs ± 26.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Making a python alternative that's already more than 2x faster

In [173]:
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_info import Race
from ares.dicts.does_not_use_larva import DOES_NOT_USE_LARVA

In [174]:
def unit_pending(unit_type: UnitTypeId) -> int:
    trained_from = UNIT_TRAINED_FROM[unit_type]
    if bot.race != Race.Zerg or unit_type == UnitTypeId.QUEEN:
        return len(
            [
                s
                for s in bot.structures
                if s.orders and s.type_id in trained_from
                and s.orders[0].ability.button_name.upper() == UnitTypeId.MARINE.name
            ]
        )
    # check eggs and cocoons for Zerg
    else:
        if type_id in DOES_NOT_USE_LARVA:
            if type_id == UnitTypeId.LURKERMP:
                return len(bot.own_units(UnitTypeId.LURKERMPEGG))
            elif type_id == UnitTypeId.OVERSEER:
                return len(bot.own_units(UnitTypeId.OVERLORDCOCOON))
            return len(bot.own_units(UnitTypeId[f"{type_id.name}COCOON"]))
        else:
            return len(
                [
                    egg
                    for egg in self.eggs
                    if egg.orders[0].ability.button_name.upper() == type_id.name
                ]
            )

In [175]:
%timeit unit_pending(UnitTypeId.MARINE)

1.17 µs ± 1.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [176]:
%%cython

from sc2.ids.unit_typeid import UnitTypeId
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_info import Race
from ares.dicts.does_not_use_larva import DOES_NOT_USE_LARVA

cimport cython

cpdef unsigned int cy_unit_pending(object unit_type, object bot):
    cdef:
        unsigned int num_pending = 0
        unsigned int len_units, x
        set trained_from = UNIT_TRAINED_FROM[unit_type]
        object units_collection, unit

    if bot.race == Race.Zerg and unit_type != UnitTypeId.QUEEN:
        if unit_type in DOES_NOT_USE_LARVA:
            units_collection = bot.own_units
            len_units = len(bot.own_units)
            trained_from = {UnitTypeId[f"{unit_type.name}COCOON"]}
            if unit_type == UnitTypeId.LURKERMP:
                trained_from = {UnitTypeId.LURKERMPEGG}
            elif unit_type == UnitTypeId.OVERSEER:
                trained_from = {UnitTypeId.OVERLORDCOCOON}
            
            for x in range(len_units):
                unit = units_collection[x]
                if unit.type_id in trained_from:
                    num_pending += 1
            return num_pending
        # unit will be pending in eggs
        else:
            units_collection = bot.eggs
            len_units = len(units_collection)
            for x in range(len_units):
                egg = units_collection[x]
                if egg.orders and egg.orders[0].ability.button_name.upper() == unit_type.name:
                    num_pending += 1
            return num_pending

    # all other units, check the structures they are built from
    else:
        units_collection = bot.structures
        len_units = len(units_collection)
        for x in range(len_units):
            structure = units_collection[x]
            if structure.orders and structure.orders[0].ability.button_name.upper() == unit_type.name:
                num_pending += 1
        return num_pending

In [177]:
%timeit cy_unit_pending(UnitTypeId.MARINE, bot)

406 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Cython version 6-7 times faster with a warning that this doesn't factor in warp ins

Something similar may be done for structures, but for this we rely on a building tracker which is not present in this notebook

# Alternative to `python-sc2`'s `units.in_attack_range`

In [178]:
from sc2.units import Units
_units = Units([u for u in units if not u.is_mineral_field and not u.is_vespene_geyser and u not in bot.destructables], bot)
bigger_units = Units([u for u in units if not u.is_mineral_field and not u.is_vespene_geyser and u not in bot.destructables], bot)
bigger_units.extend(_units)
bigger_units.extend(_units)

In [179]:
unit = _units[-6]

In [180]:
%timeit _units.in_attack_range_of(unit)

30.4 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [181]:
_units.in_attack_range_of(unit)

[Unit(name='CommandCenter', tag=4345298945), Unit(name='SCV', tag=4347396097)]

In [182]:
%timeit bigger_units.in_attack_range_of(unit)

86.8 µs ± 683 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [201]:
%%cython

import cython
from sc2.ids.unit_typeid import UnitTypeId
from ares.dicts.unit_data import UNIT_DATA

UNIT_DATA_INT_KEYS = {k.value: v for k, v in UNIT_DATA.items()}

cdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef list cy_in_attack_range(object unit, object units, double bonus_distance = 0.0):
    if not unit.can_attack:
        return []

    cdef:
        unsigned int x, len_units, type_id_int
        double dist, air_range, ground_range, radius, other_u_radius
        (float, float) unit_pos, other_unit_pos
        bint other_unit_flying, can_shoot_air, can_shoot_ground

    can_shoot_air = unit.can_attack_air
    can_shoot_ground = unit.can_attack_ground
    len_units = len(units)
    returned_units = []
    unit_pos = unit.position
    radius = unit.radius
    air_range = unit.air_range
    ground_range = unit.ground_range

    for x in range(len_units):
        u = units[x]
        type_id_int = unit._proto.unit_type
        unit_data = UNIT_DATA_INT_KEYS.get(type_id_int, None)
        if unit_data:
            other_unit_flying = unit_data["flying"]
            other_unit_pos = u.position
            other_u_radius = u.radius
            dist = cy_distance_to(unit_pos, other_unit_pos)
    
            if can_shoot_air and (other_unit_flying or type_id_int == 4):
                if dist <= air_range + radius + other_u_radius + bonus_distance:
                    returned_units.append(u)
                    # already added, no need to attempt logic below
                    continue
    
            if can_shoot_ground and not other_unit_flying:
                if dist <= ground_range + radius + other_u_radius + bonus_distance:
                    returned_units.append(u)

    return returned_units         

In [184]:
%timeit cy_in_attack_range(unit, _units)

7.28 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [185]:
cy_in_attack_range(unit, _units)

[Unit(name='CommandCenter', tag=4345298945), Unit(name='SCV', tag=4347396097)]

In [186]:
%timeit cy_in_attack_range(unit, bigger_units)

20.7 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Calculate if attack is ready

## First python implementation

In [187]:
from sc2.ids.unit_typeid import UnitTypeId as UnitID
from sc2.unit import Unit
from ares.dicts.turn_rate import TURN_RATE
import math

def angle_to(from_pos: Point2, to_pos: Point2) -> float:
    """Angle from point to other point in radians"""
    return math.atan2(to_pos.y - from_pos.y, to_pos.x - to_pos.x)

def angle_diff(a, b) -> float:
    """Absolute angle difference between 2 angles"""
    if a < 0:
        a += math.pi * 2
    if b < 0:
        b += math.pi * 2
    return math.fabs(a - b)

def get_turn_speed(unit) -> float:
    """Returns turn speed of unit in radians"""
    if unit.type_id in TURN_RATE:
        return TURN_RATE[unit.type_id] * 1.4 * math.pi / 180

def range_vs_target(unit, target) -> float:
    """Get the range of a unit to a target."""
    if unit.can_attack_air and target.is_flying:
        return unit.air_range
    else:
        return unit.ground_range

def attack_ready(bot, unit: Unit, target: Unit) -> bool:
    """
    Determine whether the unit can attack the target by the time the unit faces the target.
    Thanks Sasha for writing this out.
    """
    # fix for units, where this method returns False so the unit moves
    # but the attack animation is still active, so the move command cancels the attack
    # need to think of a better fix later, but this is better then a unit not attacking
    # and still better then using simple weapon.cooldown == 0 micro
    type_id = unit.type_id
    if unit.weapon_cooldown > 7 and type_id == UnitID.HYDRALISK:
        return True
    # prevents crash, since unit can't move
    if type_id == UnitID.LURKERMPBURROWED:
        return True
    if not unit.can_attack:
        return False
    # Time elapsed per game step
    step_time: float = bot.client.game_step / 22.4

    # Time it will take for unit to turn to face target
    angle: float = angle_diff(
        unit.facing, angle_to(unit.position, target.position)
    )
    turn_time: float = angle / get_turn_speed(unit)

    # Time it will take for unit to move in range of target
    distance = (
        unit.position.distance_to(target)
        - unit.radius
        - target.radius
        - range_vs_target(unit, target)
    )
    distance = max(0, distance)
    move_time = distance / ((unit.real_speed + 1e-16) * 1.4)

    return step_time + turn_time + move_time >= unit.weapon_cooldown / 22.4

In [188]:
target = cy_in_attack_range(unit, _units)[0]

In [189]:
attack_ready(bot, unit, target)

True

In [190]:
%timeit attack_ready(bot, unit, target)

5.66 µs ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Work out where any bottlenecks are

In [191]:
%lprun -f attack_ready attack_ready(bot, unit, target)

Timer unit: 1e-07 s

Total time: 0.0001041 s
File: C:\Users\Tom\AppData\Local\Temp\ipykernel_25596\1118029139.py
Function: attack_ready at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                           def attack_ready(bot, unit: Unit, target: Unit) -> bool:
    31                                               """
    32                                               Determine whether the unit can attack the target by the time the unit faces the target.
    33                                               Thanks Sasha for writing this out.
    34                                               """
    35                                               # fix for units, where this method returns False so the unit moves
    36                                               # but the attack animation is still active, so the move command cancels the attack
    37                                               # need to think of a better fix

In [192]:
%%cython

from sc2.ids.unit_typeid import UnitTypeId as UnitID
from sc2.unit import Unit
from ares.dicts.turn_rate import TURN_RATE
from ares.dicts.unit_data import UNIT_DATA
UNIT_DATA_INT_KEYS = {k.value: v for k, v in UNIT_DATA.items()}
TURN_RATE_INT_KEYS = {k.value: v for k, v in TURN_RATE.items()}
import math
from libc.math cimport atan2, fabs, pi

cdef double cy_distance_to(
        (float, float) p1,
        (float, float) p2
):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

cdef double angle_to((float, float) from_pos, (float, float) to_pos):
    """Angle from point to other point in radians"""
    return atan2(to_pos[1] - from_pos[1], to_pos[0] - to_pos[0])

cdef double angle_diff(double a, double b):
    """Absolute angle difference between 2 angles"""
    if a < 0:
        a += pi * 2
    if b < 0:
        b += pi * 2
    return fabs(a - b)

cdef double get_turn_speed(unit, unsigned int unit_type_int):
    """Returns turn speed of unit in radians"""
    cdef double turn_rate

    turn_rate = TURN_RATE_INT_KEYS.get(unit_type_int, None)
    if turn_rate:
        return turn_rate * 1.4 * pi / 180

cdef double range_vs_target(unit, target):
    """Get the range of a unit to a target."""
    if unit.can_attack_air and target.is_flying:
        return unit.air_range
    else:
        return unit.ground_range

cpdef bint cy_attack_ready(bot, unit, target):
    """
    Determine whether the unit can attack the target by the time the unit faces the target.
    Thanks Sasha for writing this out.
    """
    cdef:
        unsigned int unit_type_int = unit._proto.unit_type
        unsigned int weapon_cooldown
        double angle, distance, move_time, step_time, turn_time, unit_speed
        (float, float) unit_pos
        (float, float) target_pos

    # fix for units, where this method returns False so the unit moves
    # but the attack animation is still active, so the move command cancels the attack
    # need to think of a better fix later, but this is better then a unit not attacking
    # and still better then using simple weapon.cooldown == 0 micro
    weapon_cooldown = unit.weapon_cooldown
    if weapon_cooldown > 7 and unit_type_int == 91:  # 91 == UnitID.HYDRALISK
        return True
    # prevents crash, since unit can't move
    if unit_type_int == 91:  # == UnitID.LURKERMPBURROWED
        return True
    if not unit.can_attack:
        return False
    # Time elapsed per game step
    step_time = bot.client.game_step / 22.4

    unit_pos = unit.position
    target_pos = target.position
    # Time it will take for unit to turn to face target
    angle = angle_diff(
        unit.facing, angle_to(unit_pos, target_pos)
    )
    turn_time = angle / get_turn_speed(unit, unit_type_int)

    # Time it will take for unit to move in range of target
    distance = (
        cy_distance_to(unit_pos, target_pos)
        - unit.radius
        - target.radius
        - range_vs_target(unit, target)
    )
    distance = max(0, distance)
    unit_speed = (unit.real_speed + 1e-16) * 1.4
    move_time = distance / unit_speed

    return step_time + turn_time + move_time >= weapon_cooldown / 22.4

In [193]:
%timeit cy_attack_ready(bot, unit, target)

1.46 µs ± 5.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [194]:
cy_attack_ready(bot, unit, target)

True

# Pick enemy target

In [195]:
def pick_enemy_target(enemies: Units) -> Unit:
    """For best enemy target from the provided enemies
    TODO: If there are multiple units that can be killed, pick the highest value one
        Unit parameter to allow for this in the future

    For now this returns the lowest health enemy
    """
    return min(
        enemies,
        key=lambda e: (e.health + e.shield, e.tag),
    )

In [196]:
pick_enemy_target(units)

Unit(name='MineralField', tag=8906342403)

In [197]:
%timeit pick_enemy_target(units)

115 µs ± 766 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [198]:
%%cython

cpdef object cy_pick_enemy_target(enemies):
    """For best enemy target from the provided enemies
    TODO: If there are multiple units that can be killed, pick the highest value one
        Unit parameter to allow for this in the future

    For now this returns the lowest health enemy
    """
    cdef:
        object returned_unit
        unsigned int num_enemies, x
        double lowest_health, total_health
    
    num_enemies = len(enemies)
    returned_unit = enemies[0]
    lowest_health = 999.9
    for x in range(num_enemies):
        unit = enemies[x]
        total_health = unit.health + unit.shield
        if total_health < lowest_health:
            lowest_health = total_health
            returned_unit = unit

    return unit

In [199]:
%timeit cy_pick_enemy_target(units)

70.5 µs ± 818 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [200]:
cy_pick_enemy_target(units)

Unit(name='LabMineralField', tag=4307025921)